# Лабораторная работа №4

## Подготовка входных данных

Импортируем необходимые библиотеки.

In [4]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [6]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [8]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [10]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [12]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [14]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методами наивного Байеса. В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем библиотеку sklearn.naive_bayes для классификации датасета методами наивного Байеса.

In [20]:
import sklearn.naive_bayes

### Гауссовский наивный Байес

Создаём объект класса GaussianNB, реализующий метод гауссовского наивного Байеса.

In [23]:
gaussian_classifier = sklearn.naive_bayes.GaussianNB()

Определяем параметры, которые мы хотим оптимизировать для GaussianNB.
'var_smoothing': Это параметр, который контролирует сглаживание дисперсии признаков в методе Гауссовского наивного Байеса. Помогает избежать переобучения или недообучения модели.

In [25]:
gaussian_params ={
    'var_smoothing': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [27]:
gaussian_grid = GridSearchCV(gaussian_classifier, gaussian_params, cv=5, n_jobs=-1)
gaussian_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.7185882...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [29]:
best_gaussian_model = sklearn.naive_bayes.GaussianNB(**gaussian_grid.best_params_)
best_gaussian_model.fit(X_train, y_train)

GaussianNB(var_smoothing=0.49770235643321137)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [31]:
gaussian_predicted = best_gaussian_model.predict(X_test)

Вывод отчёта.

In [33]:
print('Параметры:', gaussian_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, gaussian_predicted, digits=5))

Параметры: {'var_smoothing': 0.49770235643321137}
Оценка:
               precision    recall  f1-score   support

       False    0.84243   0.94593   0.89119      9117
        True    0.65692   0.36918   0.47271      2557

    accuracy                        0.81960     11674
   macro avg    0.74968   0.65755   0.68195     11674
weighted avg    0.80180   0.81960   0.79952     11674



### Мультиномиальный наивный Байес

Создаём объект класса MultinomialNB, реализующий метод мультиноманального наивного Байеса.

In [36]:
classifier = sklearn.naive_bayes.MultinomialNB()

Определяем параметры, которые мы хотим оптимизировать для MultinomialNB.
'alpha': Это параметр, который контролирует сглаживание в модели мультиномиального наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [38]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [40]:
multinomial_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
multinomial_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [42]:
best_model = sklearn.naive_bayes.MultinomialNB(**multinomial_grid.best_params_)
best_model.fit(X_train, y_train)

MultinomialNB(alpha=1e-10)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [44]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [46]:
print('Параметры:', multinomial_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': 1e-10}
Оценка:
               precision    recall  f1-score   support

       False    0.78332   0.99923   0.87820      9117
        True    0.84091   0.01447   0.02845      2557

    accuracy                        0.78354     11674
   macro avg    0.81211   0.50685   0.45332     11674
weighted avg    0.79593   0.78354   0.69208     11674



### Комплементарный наивный Байес

Создаём объект класса ComplementNB, реализующий метод комплементарного наивного Байеса.

In [49]:
classifier = sklearn.naive_bayes.ComplementNB()

Определяем параметры, которые мы хотим оптимизировать для ComplementNB.
'alpha': Это параметр, который контролирует сглаживание в модели комплементарного наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [51]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [53]:
complement_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
complement_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07,...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [55]:
best_model = sklearn.naive_bayes.ComplementNB(**complement_grid.best_params_)
best_model.fit(X_train, y_train)

ComplementNB(alpha=13848.863713938747)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [57]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [59]:
print('Параметры:', complement_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': 13848.863713938747}
Оценка:
               precision    recall  f1-score   support

       False    0.81423   0.95042   0.87707      9117
        True    0.56202   0.22683   0.32321      2557

    accuracy                        0.79193     11674
   macro avg    0.68812   0.58863   0.60014     11674
weighted avg    0.75898   0.79193   0.75575     11674



### Бернуллиевский наивный Байес

Создаём объект класса BernoulliNB, реализующий метод бернулливского наивного Байеса.

In [62]:
classifier = sklearn.naive_bayes.BernoulliNB()

Определяем параметры, которые мы хотим оптимизировать для BernoulliNB.
'alpha': Это параметр, который контролирует сглаживание в модели бернуллиевского наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [64]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [66]:
bernoulli_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
bernoulli_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07, 4...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [68]:
best_model = sklearn.naive_bayes.BernoulliNB(**bernoulli_grid.best_params_)
best_model.fit(X_train, y_train)

BernoulliNB(alpha=2154.4346900318865)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [70]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [72]:
print('Параметры:', bernoulli_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': 2154.4346900318865}
Оценка:
               precision    recall  f1-score   support

       False    0.78098   0.99967   0.87689      9117
        True    0.25000   0.00039   0.00078      2557

    accuracy                        0.78079     11674
   macro avg    0.51549   0.50003   0.43884     11674
weighted avg    0.66468   0.78079   0.68500     11674



### Категориальный наивный Байес

Создаём объект класса CategoricalNB, реализующий метод категориального наивного Байеса.

In [75]:
classifier = sklearn.naive_bayes.CategoricalNB()

Определяем параметры, которые мы хотим оптимизировать для CategoricalNB.
* 'alpha': Это параметр, который контролирует сглаживание в модели бернуллиевского наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.
* 'min_categories': Этот параметр устанавливает минимальное количество категорий, которое должно быть в каждом признаке, чтобы этот признак использовался в модели.

In [77]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100),
    'min_categories':[2]
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [79]:
categorical_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
categorical_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10]),
                         'min_categories': [2]})

Обучаем модель на тренировочных данных.

In [81]:
best_model = sklearn.naive_bayes.CategoricalNB(**categorical_grid.best_params_)
best_model.fit(X_train, y_train)

CategoricalNB(alpha=1e-10, min_categories=2)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [83]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [85]:
print('Параметры:', categorical_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': 1e-10, 'min_categories': 2}
Оценка:
               precision    recall  f1-score   support

       False    0.79015   0.97631   0.87342      9117
        True    0.47188   0.07548   0.13014      2557

    accuracy                        0.77900     11674
   macro avg    0.63101   0.52589   0.50178     11674
weighted avg    0.72044   0.77900   0.71062     11674

